In [9]:
import psycopg2 as pg2
import pandas as pd

### Calcule a média por nome e sobrenome do ator da seguintes variáveis:
- rental_duration
- rental_rate
- length
- replacement_cost

In [10]:
conn = pg2.connect(host='localhost', port=5432, dbname='dvdrental',
                  user='postgres', password='senha')
cur = conn.cursor()

In [11]:
cur.execute('''SELECT 
first_name,
last_name,
AVG(rental_duration) AS avg_rental_duration,
AVG(rental_rate) AS avg_rental_rate,
AVG(length) AS avg_length,
AVG(replacement_cost) AS avg_replacement_cost
FROM 
(SELECT * FROM actor AS x
LEFT JOIN film_actor AS y
ON x.actor_id = y.actor_id
LEFT JOIN film AS z
ON y.film_id = z.film_id
) AS sub
GROUP BY 
first_name,
last_name
ORDER BY 
avg_replacement_cost;
''')

In [12]:
conn.commit()
data = cur.fetchall()
colnames = [desc.name for desc in cur.description]

In [13]:
df = pd.DataFrame(data, columns=colnames)
df.head()

,first_name,last_name,avg_rental_duration,avg_rental_rate,avg_length,avg_replacement_cost
0,Chris,Depp,5.2500000000000000,3.7900000000000000,106.0500000000000000,17.0900000000000000
1,Sissy,Sobieski,5.1666666666666667,2.7677777777777778,130.8333333333333333,17.6566666666666667
2,Mena,Temple,5.2666666666666667,3.0566666666666667,110.0000000000000000,17.6566666666666667
3,Kirsten,Paltrow,4.6666666666666667,3.4344444444444444,110.7037037037037037,17.9900000000000000
4,Mae,Hoffman,4.8571428571428571,2.4900000000000000,116.1071428571428571,17.9900000000000000


### Calcule a soma de amount (preço total do aluguel) por nome, sobrenome e email do cliente (customer) e indique os 10 clientes que mais gastaram e os 10 que menos gastaram.

In [14]:
cur.execute('''SELECT
x.first_name,
x.last_name,
x.email,
SUM(y.amount) AS total_spent
FROM customer AS x
LEFT JOIN payment AS y
ON x.customer_id = y.customer_id
GROUP BY
x.first_name,
x.last_name,
x.email
ORDER BY
total_spent DESC
LIMIT 10;
''')

In [15]:
conn.commit()
mais_gataram = cur.fetchall()
colnames = [desc.name for desc in cur.description]

In [16]:
df = pd.DataFrame(mais_gataram, columns=colnames)
df.head()

,first_name,last_name,email,total_spent
0,Eleanor,Hunt,eleanor.hunt@sakilacustomer.org,211.55
1,Karl,Seal,karl.seal@sakilacustomer.org,208.58
2,Marion,Snyder,marion.snyder@sakilacustomer.org,194.61
3,Rhonda,Kennedy,rhonda.kennedy@sakilacustomer.org,191.62
4,Clara,Shaw,clara.shaw@sakilacustomer.org,189.60


In [17]:
cur.execute('''SELECT
x.first_name,
x.last_name,
x.email,
SUM(y.amount) AS total_spent
FROM customer AS x
LEFT JOIN payment AS y
ON x.customer_id = y.customer_id
GROUP BY
x.first_name,
x.last_name,
x.email
ORDER BY
total_spent ASC
LIMIT 10;
''')

In [18]:
conn.commit()
menos_gataram = cur.fetchall()
colnames = [desc.name for desc in cur.description]

In [19]:
df = pd.DataFrame(menos_gataram, columns=colnames)
df.head()

,first_name,last_name,email,total_spent
0,Brian,Wyman,brian.wyman@sakilacustomer.org,27.93
1,Leona,Obrien,leona.obrien@sakilacustomer.org,32.90
2,Caroline,Bowman,caroline.bowman@sakilacustomer.org,37.87
3,Anthony,Schwab,anthony.schwab@sakilacustomer.org,47.85
4,Tiffany,Jordan,tiffany.jordan@sakilacustomer.org,49.88


In [20]:
cur.close()
conn.close()